In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

import collections
import copy
import sys
sys.stdout.flush()

from processing_functions import *

from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['mathtext.fontset'] = 'dejavuserif'
rcParams['mathtext.rm'] = 'serif'
rcParams['mathtext.it'] = 'serif:italic'
rcParams['mathtext.bf'] = 'serif:bold'
rcParams['axes.titlepad'] = 12

### Calculate inverse covariance matrices by combination.

In [2]:
jkcov = np.load('Run_30_bins_for_all/jk_19p0.npz')['jkcov']
fidreal = np.load('Run_30_bins_for_all/fidreal_19p0.npz')['func']
fidrandom = np.load('Run_30_bins_for_all/fidrandom_19p0.npz')

covtot = calc_covariance(jkcov,fidreal,fidrandom)
cov_cut = apply_to_dict(cov_clean,cut_by_func_2D(covtot))
nzidx_cut = apply_to_dict(cov_clean,cut_by_func_2D(covtot),return_idx=1)
invcov_cut = apply_to_dict(inv_cov,cov_cut,rcond=1e-4)

### Load perturbed datasets, use one for smoothing test, find derivatives for all.

In [3]:
pertfunc_dict, pertparam_dict = load_pert('-19.0')

In [4]:
pertfunc_eg = pertfunc_dict['06837']
pertparam_eg = pertparam_dict['06837']
func_cut_eg = apply_to_dict(pert_clean,cut_by_func_1D(pertfunc_eg,axis=1),nzidx_cut)

In [5]:
def get_alphas(combkey):
    return iterated_alpha(func_cut_eg[combkey],invcov_cut[combkey],pertparam_eg,p19p0,oneside_ranges[0])

In [6]:
from concurrent.futures import ProcessPoolExecutor as Pool

In [14]:
nproc = 55
alphas_cut = {}
keylist = list(func_cut_eg.keys())
with Pool(nproc) as pool:
    for i, output_data in enumerate(pool.map(get_alphas, keylist)):
        alphas_cut[keylist[i]] = output_data

Process Process-19:
Process Process-51:
Process Process-45:
Process Process-35:
Process Process-13:
Process Process-22:
Process Process-27:
Process Process-23:
Process Process-53:
Process Process-20:
Process Process-2:
Process Process-3:
Process Process-8:
Process Process-38:
Process Process-32:
Process Process-1:
Process Process-18:
Process Process-29:
Process Process-26:
Process Process-21:
Process Process-49:
Process Process-25:
/home/kuw8/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in matrix(t1[1:((3 * d + 8) * nvm)], nrow = nvm) : Interrupted

  warnings.warn(x, RRuntimeWarning)
/home/kuw8/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in lfproc(x, y, weights = weights, cens = cens, base = base, geth = geth,  : 
  Interrupted

  warnings.warn(x, RRuntimeWarning)
/home/kuw8/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in simpleLoess(y, x,

KeyboardInterrupt: 

  File "/home/kuw8/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/kuw8/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/kuw8/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/kuw8/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/kuw8/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/kuw8/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/kuw8/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/kuw8/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/kuw8/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/kuw8/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/kuw8/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/kuw8/an

In [15]:
alphas_cut

{'ca': array([ 0.21149176,  0.28642296,  0.20833589,  0.2679786 ,  0.20032073,
         0.20461184,  0.21062566]),
 'dar': array([ 0.22288952,  0.2104129 ,  0.20862776,  0.22465447,  0.20032073,
         0.20019878,  0.22248997]),
 'va': array([ 0.3989988 ,  0.32665434,  0.20862776,  0.68639206,  0.20032073,
         0.20461184,  0.39364651]),
 'vc': array([ 0.21149176,  0.28642296,  0.20833589,  0.53437818,  0.20873776,
         0.20971624,  0.29316831])}